## **Practical SQL for Data Analysis**
### What you can do *together with* Pandas



In [ ]:
!pip install --quiet duckdb

     |████████████████████████████████| 8.5MB 8.5MB/s 


In [ ]:
import pandas as pd
import numpy as np

import sqlite3
import duckdb
import time

**Preparation**

Download the data and set up the Pandas data frames. We read the data into a Pandas DataFrame using DuckDB's built-in Parquet reader.

In [ ]:
!wget -q https://github.com/cwida/duckdb-data/releases/download/v1.0/lineitemsf1.snappy.parquet

In [ ]:
!wget -q https://github.com/cwida/duckdb-data/releases/download/v1.0/orders.parquet

In [ ]:
lineitem = duckdb.query("SELECT * FROM 'lineitemsf1.snappy.parquet'").to_df()
orders = duckdb.query("SELECT * FROM 'orders.parquet'").to_df()

In [ ]:
con = duckdb.connect()
con.execute('PRAGMA threads=2')


def timeit(fun, name):
    import time
    start_time = time.monotonic()
    fun()
    return [name, time.monotonic() - start_time]


def plot_results(results, title):
    df = pd.DataFrame.from_dict({
        'name': [x[0] for x in results],
        'time': [x[1] for x in results]
    })
    print(title)
    print(df)

**Ungrouped Aggregates**

This performs a simple set of ungrouped aggregates (sum, min, max, avg) over a column without any filters or other complex operations.





In [ ]:
ungrouped_aggregate = '''
    SELECT SUM(l_extendedprice), MIN(l_extendedprice), MAX(l_extendedprice), AVG(l_extendedprice) FROM lineitem
'''

def duckdb_ungrouped_aggregate(d_con):
	print(d_con.query(ungrouped_aggregate).to_df())

def duckdb_ungrouped_aggregate_1t():
	duckdb_ungrouped_aggregate(duckdb)

def duckdb_ungrouped_aggregate_2t():
	duckdb_ungrouped_aggregate(con)

def pandas_ungrouped_aggregate():
  result = lineitem.groupby(
    ['l_returnflag', 'l_linestatus']
  ).agg(
    Sum=('l_extendedprice', 'sum'),
    Min=('l_extendedprice', 'min'),
    Max=('l_extendedprice', 'max'),
    Avg=('l_extendedprice', 'mean')
  )
  print(result)
	# print(lineitem.agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))

ua_results = []
ua_results.append(timeit(duckdb_ungrouped_aggregate_1t, 'DuckDB (1T)'))
ua_results.append(timeit(duckdb_ungrouped_aggregate_2t, 'DuckDB (2T)'))
ua_results.append(timeit(pandas_ungrouped_aggregate, 'Pandas'))
plot_results(ua_results, 'Ungrouped Aggregate')


   sum(l_extendedprice)  ...  avg(l_extendedprice)
0          2.295773e+11  ...          38255.138485

[1 rows x 4 columns]
   sum(l_extendedprice)  ...  avg(l_extendedprice)
0          2.295773e+11  ...          38255.138485

[1 rows x 4 columns]
                                    Sum    Min       Max           Avg
l_returnflag l_linestatus                                             
A            F             5.658655e+10  904.0  104949.5  38273.129735
N            F             1.487505e+09  920.0  104049.5  38284.467761
             O             1.149352e+11  901.0  104749.5  38248.015609
R            F             5.656804e+10  904.0  104899.5  38250.854626
Ungrouped Aggregate
          name      time
0  DuckDB (1T)  0.073622
1  DuckDB (2T)  0.056383
2       Pandas  0.894141


**Grouped Aggregates**

This performs the same set of aggregates, but this time grouped by two other columns (*l_returnflag* and *l_linestatus*).



In [ ]:
grouped_aggregate = '''
SELECT l_returnflag,
       l_linestatus,
       SUM(l_extendedprice),
       MIN(l_extendedprice),
       MAX(l_extendedprice),
       AVG(l_extendedprice)
FROM lineitem
GROUP BY l_returnflag,
         l_linestatus
'''

def duckdb_grouped_aggregate(d_con):
    print(d_con.query(grouped_aggregate).to_df())

def duckdb_grouped_aggregate_1t():
	  duckdb_grouped_aggregate(duckdb)

def duckdb_grouped_aggregate_2t():
	  duckdb_grouped_aggregate(con)

def pandas_grouped_aggregate():
	  print(lineitem.groupby(['l_returnflag', 'l_linestatus']).agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))

results = []
results.append(timeit(duckdb_grouped_aggregate_1t, 'DuckDB (1T)'))
results.append(timeit(duckdb_grouped_aggregate_2t, 'DuckDB (2T)'))
results.append(timeit(pandas_grouped_aggregate, 'Pandas'))
plot_results(results, 'Grouped Aggregate')

  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38248.015609
1            R            F  ...              104899.5          38250.854626
2            A            F  ...              104949.5          38273.129735
3            N            F  ...              104049.5          38284.467761

[4 rows x 6 columns]
  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38248.015609
1            R            F  ...              104899.5          38250.854626
2            A            F  ...              104949.5          38273.129735
3            N            F  ...              104049.5          38284.467761

[4 rows x 6 columns]
                                    Sum    Min       Max           Avg
l_returnflag l_linestatus                                             
A            F             5.658655e+10  904

**Grouped Aggregate with a Filter**

This benchmark performs a grouped aggregate with a filter over the shipdate column.

As Pandas does not perform any projection pushdown, we include a version where we manually perform the projection pushdown by filtering only the columns we actually need before running the filter and aggregate.

This optimization is performed automatically in DuckDB by the query optimizer.

In [ ]:
def duckdb_grouped_aggregate_filter(d_con):
	  print(d_con.query('''
SELECT l_returnflag,
       l_linestatus,
       SUM(l_extendedprice),
       MIN(l_extendedprice),
       MAX(l_extendedprice),
       AVG(l_extendedprice)
FROM lineitem
WHERE
    l_shipdate <= DATE '1998-09-02'
GROUP BY l_returnflag,
         l_linestatus
''').to_df())

def duckdb_grouped_aggregate_filter_1t():
	  duckdb_grouped_aggregate_filter(duckdb)

def duckdb_grouped_aggregate_filter_2t():
	  duckdb_grouped_aggregate_filter(con)

def pandas_grouped_aggregate_filter():
    filtered_df = lineitem[lineitem['l_shipdate'] < "1998-09-02"]
    print(filtered_df.groupby(['l_returnflag', 'l_linestatus']).agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))

def pandas_grouped_aggregate_filter_projection_pushdown():
    pushed_down_df = lineitem[['l_shipdate', 'l_returnflag', 'l_linestatus', 'l_extendedprice']]
    filtered_df = pushed_down_df[pushed_down_df['l_shipdate'] < "1998-09-02"]
    print(filtered_df.groupby(['l_returnflag', 'l_linestatus']).agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))

results = []
results.append(timeit(duckdb_grouped_aggregate_filter_1t, 'DuckDB (1T)'))
results.append(timeit(duckdb_grouped_aggregate_filter_2t, 'DuckDB (2T)'))
results.append(timeit(pandas_grouped_aggregate_filter, 'Pandas'))
results.append(timeit(pandas_grouped_aggregate_filter_projection_pushdown, 'Pandas (manual pushdown)'))
plot_results(results, 'Grouped Aggregate + Filter')


  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38249.117989
1            R            F  ...              104899.5          38250.854626
2            A            F  ...              104949.5          38273.129735
3            N            F  ...              104049.5          38284.467761

[4 rows x 6 columns]
  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38249.117989
1            A            F  ...              104949.5          38273.129735
2            R            F  ...              104899.5          38250.854626
3            N            F  ...              104049.5          38284.467761

[4 rows x 6 columns]
                                    Sum    Min       Max           Avg
l_returnflag l_linestatus                                             
A            F             5.658655e+10  904

**Grouped Aggregate with Join and Filter**

In this benchmark we expand on the previous benchmark by including a join and a filter on the joined-on table.

Note that the naive version in Pandas is extremely slow, as it performs a full join of the entire table including all the columns that are not used and all the rows that will be filtered out. For that reason we have included a separate benchmark in which we have manually optimized the Pandas code by pushing down the projections and the filters.

These optimizations are performed automatically in DuckDB by the query optimizer.

In [ ]:

# projection & filter on lineitem table
lineitem_projected = lineitem[
    ['l_shipdate',
    'l_orderkey',
    'l_linestatus',
    'l_returnflag',
    'l_extendedprice']
]
lineitem_filtered = lineitem_projected[lineitem_projected['l_shipdate'] < "1998-09-02"]
# projection and filter on order table
orders_projected = orders[
    ['o_orderkey',
    'o_orderstatus']
]
orders_filtered = orders_projected[orders_projected['o_orderstatus'] == 'O']
# perform the join
merged = lineitem_filtered.merge(
    orders_filtered,
    left_on='l_orderkey',
    right_on='o_orderkey'
)
# perform the aggregate
result = merged.groupby(
    ['l_returnflag', 'l_linestatus']
).agg(
    Sum=('l_extendedprice', 'sum'),
    Min=('l_extendedprice', 'min'),
    Max=('l_extendedprice', 'max'),
    Avg=('l_extendedprice', 'mean')
)

In [ ]:
result

,,Sum,Min,Max,Avg
l_returnflag,l_linestatus,,,,
N,O,1.080448e+11,901.0,104749.5,38250.662806


In [ ]:
def duckdb_grouped_aggregate_filter_join(d_con):
		print(d_con.query('''
SELECT l_returnflag,
       l_linestatus,
       sum(l_extendedprice),
       min(l_extendedprice),
       max(l_extendedprice),
       avg(l_extendedprice)
FROM lineitem lineitem
JOIN orders orders ON (l_orderkey=o_orderkey)
WHERE l_shipdate <= DATE '1998-09-02'
  AND o_orderstatus='O'
GROUP BY l_returnflag,
         l_linestatus
''').to_df())


def duckdb_grouped_aggregate_filter_join_1t():
		duckdb_grouped_aggregate_filter_join(duckdb)


def duckdb_grouped_aggregate_filter_join_2t():
		duckdb_grouped_aggregate_filter_join(con)


def pandas_grouped_aggregate_filter_join():
		merged = lineitem.merge(orders, left_on='l_orderkey', right_on='o_orderkey')
		filtered_a = merged[merged['l_shipdate'] < "1998-09-02"]
		filtered_b = filtered_a[filtered_a['o_orderstatus'] == 'O']
		result = filtered_b.groupby(['l_returnflag', 'l_linestatus']).agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean'))
		print(result)


def pandas_grouped_aggregate_filter_join_manual_pushdown():
		lineitem_projected = lineitem[['l_shipdate', 'l_orderkey', 'l_linestatus', 'l_returnflag', 'l_extendedprice']]
		lineitem_filtered = lineitem_projected[lineitem_projected['l_shipdate'] < "1998-09-02"]
		orders_projected = orders[['o_orderkey', 'o_orderstatus']]
		orders_filtered = orders_projected[orders_projected['o_orderstatus'] == 'O']
		merged = lineitem_filtered.merge(orders_filtered, left_on='l_orderkey', right_on='o_orderkey')
		result = merged.groupby(['l_returnflag', 'l_linestatus']).agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean'))
		print(result)


results = []
results.append(timeit(duckdb_grouped_aggregate_filter_join_1t, 'DuckDB (1T)'))
results.append(timeit(duckdb_grouped_aggregate_filter_join_2t, 'DuckDB (2T)'))
results.append(timeit(pandas_grouped_aggregate_filter_join, 'Pandas'))
results.append(timeit(pandas_grouped_aggregate_filter_join_manual_pushdown, 'Pandas (manual pushdown)'))
plot_results(results, 'Grouped Aggregate Join')


  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38250.450307

[1 rows x 6 columns]
  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38250.450307

[1 rows x 6 columns]
                                    Sum    Min       Max           Avg
l_returnflag l_linestatus                                             
N            O             1.080448e+11  901.0  104749.5  38250.662806
                                    Sum    Min       Max           Avg
l_returnflag l_linestatus                                             
N            O             1.080448e+11  901.0  104749.5  38250.662806
Grouped Aggregate Join
                       name       time
0               DuckDB (1T)   0.766380
1               DuckDB (2T)   0.548619
2                    Pandas  10.665751
3  Pandas (manual pushdown)   3.418275


# Appendix A: There and back again: Transferring data from Pandas to a SQL engine and back

As Appendix A relies on the presence of an external PostgreSQL database server, the code cannot be executed in colab. The source code can be found here: https://gist.github.com/hannesmuehleisen/a95a39a1eda63aeb0ca13fd82d1ba49c


# Appendix B: PandasSQL

Note: we cannot run this on the original dataset, as colab will run out of memory and crash. Instead for the benchmark we add a sample clause to reduce the data set size to 10% of the original data set size.

In [ ]:
!pip install --quiet pandasql

In [ ]:
import pandasql as psql
pysqldf = lambda q: psql.sqldf(q, globals())


In [ ]:
lineitem_sample = duckdb.query("SELECT * FROM 'lineitemsf1.snappy.parquet' USING SAMPLE 10%").to_df()

In [ ]:
ungrouped_aggregate = '''
    SELECT SUM(l_extendedprice), MIN(l_extendedprice), MAX(l_extendedprice), AVG(l_extendedprice) FROM lineitem_sample
'''

def duckdb_ungrouped_aggregate(d_con):
		print(d_con.query(ungrouped_aggregate).to_df())

def duckdb_ungrouped_aggregate_1t():
		duckdb_ungrouped_aggregate(duckdb)

def duckdb_ungrouped_aggregate_2t():
		duckdb_ungrouped_aggregate(con)

def pandas_ungrouped_aggregate():
		print(lineitem_sample.agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))

def ungrouped_aggregate_pandasql():
  	print(pysqldf(ungrouped_aggregate))


ua_results = []
ua_results.append(timeit(duckdb_ungrouped_aggregate_1t, 'DuckDB (1T)'))
ua_results.append(timeit(duckdb_ungrouped_aggregate_2t, 'DuckDB (2T)'))
ua_results.append(timeit(pandas_ungrouped_aggregate, 'Pandas'))
ua_results.append(timeit(ungrouped_aggregate_pandasql, 'PandaSQL'))
plot_results(ua_results, 'Ungrouped Aggregate')

   sum(l_extendedprice)  ...  avg(l_extendedprice)
0          2.235921e+10  ...          38270.334424

[1 rows x 4 columns]
   sum(l_extendedprice)  ...  avg(l_extendedprice)
0          2.235921e+10  ...          38270.334424

[1 rows x 4 columns]
     l_extendedprice
Sum     2.235921e+10
Min     9.010000e+02
Max     1.048995e+05
Avg     3.827033e+04
   SUM(l_extendedprice)  ...  AVG(l_extendedprice)
0          2.235921e+10  ...          38270.334424

[1 rows x 4 columns]
Ungrouped Aggregate
          name       time
0  DuckDB (1T)   0.021568
1  DuckDB (2T)   0.018576
2       Pandas   0.017274
3     PandaSQL  28.341653


# Appendix C: Directly querying Parquet files

In the benchmarks above, we fully read the parquet files into Pandas. However, DuckDB also has the capability of directly running queries on top of Parquet files. In this appendix, we show the performance of this compared to loading the file into Python first.

You can even use the wildcard syntax to run queries on multiple Parquet files in the same folder and create a unified single-table view over them (as long as they have the same schema).


In [ ]:
# need to install pyarrow for pandas parquet reading
!pip install pyarrow

In [ ]:
# set up the view
parquet_con = duckdb.connect()
parquet_con.execute("CREATE VIEW lineitem_parquet AS SELECT * FROM 'lineitemsf1.snappy.parquet'")
parquet_con.execute("CREATE VIEW orders_parquet AS SELECT * FROM 'orders.parquet'")


# Ungrouped Aggregate

In [ ]:
ungrouped_aggregate = '''
    SELECT SUM(l_extendedprice), MIN(l_extendedprice), MAX(l_extendedprice), AVG(l_extendedprice) FROM lineitem_parquet
'''

def duckdb_parquet_query(d_con):
	  print(d_con.query(ungrouped_aggregate).to_df())

def duckdb_ungrouped_parquet_1t():
    parquet_con.execute('PRAGMA threads=1')
    duckdb_ungrouped_aggregate(parquet_con)

def duckdb_ungrouped_parquet_2t():
    parquet_con.execute('PRAGMA threads=2')
    duckdb_ungrouped_aggregate(parquet_con)

def pandas_ungrouped_aggregate():
	  print(lineitem.agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))

def pandas_ungrouped_aggregate_parquet_load():
    lineitem_pandas_parquet = pd.read_parquet('lineitemsf1.snappy.parquet')
    print(lineitem_pandas_parquet.agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))

def pandas_ungrouped_aggregate_parquet_load_pushdown():
    lineitem_pandas_parquet = pd.read_parquet('lineitemsf1.snappy.parquet', columns=['l_extendedprice'])
    print(lineitem_pandas_parquet.agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean')))


ua_results = []
ua_results.append(timeit(duckdb_ungrouped_parquet_1t, 'DuckDB (1 Thread)'))
ua_results.append(timeit(duckdb_ungrouped_parquet_2t, 'DuckDB (2 Threads)'))
ua_results.append(timeit(pandas_ungrouped_aggregate, 'Pandas'))
ua_results.append(timeit(pandas_ungrouped_aggregate_parquet_load, 'Pandas + Parquet Load'))
ua_results.append(timeit(pandas_ungrouped_aggregate_parquet_load_pushdown, 'Pandas + Parquet Load (Pushdown)'))
plot_results(ua_results, 'Ungrouped Aggregate (Parquet)')

   sum(l_extendedprice)  ...  avg(l_extendedprice)
0          2.295773e+11  ...          38255.138485

[1 rows x 4 columns]
   sum(l_extendedprice)  ...  avg(l_extendedprice)
0          2.295773e+11  ...          38255.138485

[1 rows x 4 columns]
     l_extendedprice
Sum     2.295773e+11
Min     9.010000e+02
Max     1.049495e+05
Avg     3.825514e+04
     l_extendedprice
Sum     2.295773e+11
Min     9.010000e+02
Max     1.049495e+05
Avg     3.825514e+04
     l_extendedprice
Sum     2.295773e+11
Min     9.010000e+02
Max     1.049495e+05
Avg     3.825514e+04
Ungrouped Aggregate (Parquet)
                               name      time
0                 DuckDB (1 Thread)  0.212070
1                DuckDB (2 Threads)  0.146053
2                            Pandas  0.063777
3             Pandas + Parquet Load  7.984914
4  Pandas + Parquet Load (Pushdown)  0.177484


# Grouped Aggregate with Join and Filter

In [ ]:
def duckdb_grouped_aggregate_filter_join_pq(d_con):
	  print(d_con.query('''
SELECT l_returnflag,
       l_linestatus,
       sum(l_extendedprice),
       min(l_extendedprice),
       max(l_extendedprice),
       avg(l_extendedprice)
FROM lineitem_parquet lineitem
JOIN orders_parquet orders ON (l_orderkey=o_orderkey)
WHERE l_shipdate <= DATE '1998-09-02'
  AND o_orderstatus='O'
GROUP BY l_returnflag,
         l_linestatus
''').to_df())


def duckdb_grouped_aggregate_filter_join_pq_1t():
    parquet_con.execute('PRAGMA threads=1')
    duckdb_grouped_aggregate_filter_join_pq(parquet_con)


def duckdb_grouped_aggregate_filter_join_pq_2t():
    parquet_con.execute('PRAGMA threads=2')
    duckdb_grouped_aggregate_filter_join_pq(parquet_con)


def pandas_grouped_aggregate_filter_join_pq():
    lineitem_pandas_parquet = pd.read_parquet('lineitemsf1.snappy.parquet')
    orders_pandas_parquet = pd.read_parquet('orders.parquet')
    merged = lineitem_pandas_parquet.merge(orders, left_on='l_orderkey', right_on='o_orderkey')
    filtered_a = merged[merged['l_shipdate'] < "1998-09-02"]
    filtered_b = filtered_a[filtered_a['o_orderstatus'] == 'O']
    result = filtered_b.groupby(['l_returnflag', 'l_linestatus']).agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean'))
    print(result)


def pandas_grouped_aggregate_filter_join_manual_pushdown_pq():
    lineitem_projected = pd.read_parquet('lineitemsf1.snappy.parquet', columns=['l_shipdate', 'l_orderkey', 'l_linestatus', 'l_returnflag', 'l_extendedprice'])
    orders_projected = pd.read_parquet('orders.parquet', columns=['o_orderkey', 'o_orderstatus'])
    lineitem_filtered = lineitem_projected[lineitem_projected['l_shipdate'] < "1998-09-02"]
    orders_filtered = orders_projected[orders_projected['o_orderstatus'] == 'O']
    merged = lineitem_filtered.merge(orders_filtered, left_on='l_orderkey', right_on='o_orderkey')
    result = merged.groupby(['l_returnflag', 'l_linestatus']).agg(Sum=('l_extendedprice', 'sum'), Min=('l_extendedprice', 'min'), Max=('l_extendedprice', 'max'), Avg=('l_extendedprice', 'mean'))
    print(result)



results = []
results.append(timeit(duckdb_grouped_aggregate_filter_join_pq_1t, 'DuckDB (1T)'))
results.append(timeit(duckdb_grouped_aggregate_filter_join_pq_2t, 'DuckDB (2T)'))
results.append(timeit(pandas_grouped_aggregate_filter_join_pq, 'Pandas'))
results.append(timeit(pandas_grouped_aggregate_filter_join_manual_pushdown_pq, 'Pandas (manual pushdown)'))
plot_results(results, 'Grouped Aggregate Join (Parquet)')


  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38250.450307

[1 rows x 6 columns]
  l_returnflag l_linestatus  ...  max(l_extendedprice)  avg(l_extendedprice)
0            N            O  ...              104749.5          38250.450307

[1 rows x 6 columns]
Grouped Aggregate Join (Parquet)
          name      time
0  DuckDB (1T)  1.089228
1  DuckDB (2T)  0.875342
